### GENERATE_DH

In [1]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives import serialization

class DH_pair:
    def __init__(self, dh_priv, dh_pub) -> None:
        self.dh_priv = dh_priv
        self.dh_pub = dh_pub

def GENERATE_DH():
    # Generate a new DH pair key.
    dh_priv = X25519PrivateKey.generate()
    dh_pub = dh_priv.public_key()
    
    return DH_pair(dh_priv, dh_pub)

dh_pair = GENERATE_DH()
dh_pair2 = GENERATE_DH()

print("Private key:", dh_pair.dh_priv.private_bytes(encoding=serialization.Encoding.Raw, format=serialization.PrivateFormat.Raw, encryption_algorithm=serialization.NoEncryption()))
print("Length Private key:", len(dh_pair.dh_priv.private_bytes(encoding=serialization.Encoding.Raw, format=serialization.PrivateFormat.Raw, encryption_algorithm=serialization.NoEncryption())))
print()
print("Public key:", dh_pair.dh_pub.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw))
print("Length Public key:", len(dh_pair.dh_pub.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)))


Private key: b"\xb8\xa8\xdc*\xb4\xce\x9d\x01\xa0\x83\xb6\x1da\x1c\x9a\x03\xa80'\xf1k\xfd\xe7\x8a\xae\x9e(\x85\xf5\x15\xb6\x7f"
Length Private key: 32

Public key: b')\x02\xd7\xa7F\xff\xdd\x1c\xac\xcfL\xe7\xd7V\x06\xc9\xbe4\xbe)<\xc92\xd00\x9b\xb8\tz\xb1\xd3g'
Length Public key: 32


### DH

In [2]:
def DH(dh_pair: DH_pair, dh_pub):
    # Perform DH calculation between private key and public key
    return dh_pair.dh_priv.exchange(dh_pub)

# Calcular la clave compartida usando la función DH()
dh_out = DH(dh_pair=dh_pair, dh_pub=dh_pair2.dh_pub)
dh_out2 = DH(dh_pair=dh_pair2, dh_pub=dh_pair.dh_pub)
print("Shared secret 1:", dh_out)
print("Shared secret 2:", dh_out2)

Shared secret 1: b'\x84\x16v\t\x95\xd0\x81\xc5\xfae\x90b\x88`\xa9U\x05\x1b\xc4\xb5\x83W\xdaj\x0b\xf4%`9\xf7\rD'
Shared secret 2: b'\x84\x16v\t\x95\xd0\x81\xc5\xfae\x90b\x88`\xa9U\x05\x1b\xc4\xb5\x83W\xdaj\x0b\xf4%`9\xf7\rD'


### KDF_RK

In [3]:
import secrets

rk = secrets.token_bytes(32)
print("Random",len(rk),"bytes:", rk)

Random 32 bytes: b'\xfbS!2\xdf\x99\xdf3D9\x9f\xddz\x87\xf2\xf6 \xa0\xda\xc1\xef`VC07\xf4\xd6|\x0e\x17\x90'


In [6]:
from hkdf import hkdf_expand, hkdf_extract
from binascii import unhexlify
import hashlib

def KDF_RK(rk: bytes, dh_out: bytes) -> [[bytes]*32, [bytes]*32]:
    prk = hkdf_extract(salt=rk, input_key_material=dh_out, hash=hashlib.sha512)
    key = hkdf_expand(pseudo_random_key=prk, info=b"Info KDF_RK", length=64, hash=hashlib.sha512)

    return key[:32],key[32:] # Root Key, Chain Key

rk, ck = KDF_RK(rk=rk, dh_out=dh_out)
print("Root Key:", rk)
print("Length Root Key:", len(rk), "\n")
print("Chain Key:", ck)
print("Length Chain Key:", len(ck))

Root Key: b'\xdb[\x9e\x1a*\xfb\xf7\x9epoF49\xd5\xda\xdc\xff\x0e\xa9\x95\x0ek\x82FU{\t\x04KPB\x08'
Length Root Key: 32 

Chain Key: b'D\x02\xf7\x98\xa3\xbbV\t\xea\x98a\x1b3\x03<t1\x84\x82<\xe5\xbc\xdfa\x16\xf9\xc9|\x1aZUp'
Length Chain Key: 32


### KDF_CK

In [7]:
import hmac

def KDF_CK(ck: bytes) -> [[bytes]*32, [bytes]*32]:
    new_ck = hmac.new(key=ck, msg=b'Chain Key', digestmod=hashlib.sha512)
    mk = hmac.new(key=ck, msg=b'Message Key', digestmod=hashlib.sha512)

    return new_ck.digest()[:32], mk.digest()[:32] # Chain Key, Message Key


ck, mk = KDF_CK(ck=ck)

print("Chain Key:",ck)
print("Length Chain Key:",len(ck))
print()
print("Message Key:",mk)
print("Length Message Key:",len(mk))


Chain Key: b'CK!s.\xcd\xc6\xbe\xbd\x84\x9c\xd7\xd4\x93\xf9\xa0$\xd0\x14_\x06iX\xac\x81\xfe>\xf2#n\x18]'
Length Chain Key: 32

Message Key: b'\xec\xf8\x04%\xac\x1dT\xc4\xb7\\\xf7p\xf4\xce\xdb!\xf6\xad,\xf0\xb0\x90\x03$\xaa\xa4\x80\x82E\xe82\xd1'
Length Message Key: 32


### ENCRYPT

In [31]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding  # - Import to use the padding.
from cryptography.hazmat.primitives.ciphers import (
    Cipher, algorithms, modes
)

def ENCRYPT(mk: bytes, plaintext: bytes, assotiate_data: bytes):
    # Key Derivation Function
    output_length = 80
    prk = hkdf_extract(salt=bytes(output_length), input_key_material=mk, hash=hashlib.sha512)
    exp = hkdf_expand(pseudo_random_key=prk, info=b"Info ENCRYPT/DECRYPT", length=output_length, hash=hashlib.sha512)

    sk = exp[:32] # Encryption Key
    ak = exp[32:64] # Authentication Key
    iv = exp[64:] # Initialization vector

    # Construct an AES-128-GCM Cipher object with the generated encryption key and IV.
    encryptor = Cipher(
            algorithms.AES(key=sk),
            modes.GCM(initialization_vector=iv),
            backend=default_backend()
        ).encryptor()

    padder = padding.PKCS7(128).padder()
    padder_data = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(padder_data) + encryptor.finalize()
    tag = encryptor.tag # Tag is always 16B long.

    # HMAC for authentication of the associate it data.
    ad = hmac.new(key=ak, msg=assotiate_data, digestmod=hashlib.sha512).digest() # 512b = 64B

    return tag + ad + ciphertext

enc_msg = ENCRYPT(mk=mk,plaintext=b"hola",assotiate_data=b"hola")
print(enc_msg)
print(len(enc_msg))

b'D\x8d\x04h\x86\nJN.\xff\xd7\xc03\x97p\xb29)\xaa\xd0\xf2\xa1a\xcfX\xbc\xbc\x12\xa4\xbbe\\\x05\xe2S\xe7\xc9\xd47\x94,<\x982\xedm^\x0e5\xb6\xb3E;\xcc\xad\xd9g:V\xdd\x0b\x81C\xce\xa8\xfb\xe9\xafP\xc8\xb6|&\xd0[\x18\xf0pz\xa5\x0e\xb1D\x00\x0f\x90\xaf\x89<\t\xaa\x02M\xa2\x7f3'
96


### HEADER

In [32]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PublicKey

class HEADER:
    def __init__(self, dh_pair=None, pn: int = 0, n: int = 0) -> None:
        if dh_pair is not None:
            self.dh = dh_pair.dh_pub
            self.pn = pn
            self.n = n

    def to_bytes(self):
        dh_pub_b = self.dh.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw) # 32 B
        pn_b = self.pn.to_bytes(1, byteorder='big') # 1 B
        n_b = self.n.to_bytes(1, byteorder='big') # 1 B

        return dh_pub_b + pn_b + n_b 
    
    def to_object(self, header_bytes: [bytes]*34):
        self.dh = X25519PublicKey.from_public_bytes(data=header_bytes[:32])
        self.pn = int.from_bytes(header_bytes[32:33], byteorder='big')
        self.n = int.from_bytes(header_bytes[33:34], byteorder='big')

header = HEADER(dh_pair=dh_pair, pn=4, n=3)
print(header.to_bytes())
print(header)

b')\x02\xd7\xa7F\xff\xdd\x1c\xac\xcfL\xe7\xd7V\x06\xc9\xbe4\xbe)<\xc92\xd00\x9b\xb8\tz\xb1\xd3g\x04\x03'


In [101]:


dh_pub = X25519PublicKey.from_public_bytes(data=header[:32])

### CONCAT

In [33]:
def CONCAT(ad: bytes, header: HEADER):
    return ad + header.to_bytes()

ad = b"Hola"
ad = hashlib.sha256(ad).digest()
print(len(ad))
concat = CONCAT(ad=ad, header=header)
print(concat)
h2 = HEADER()
print(type(h2))
h2.to_object(header_bytes=concat[32:])
h2.n

32
b'\xe63\xf4\xfcy\xba\xde\xa1\xdc]\xb9p\xcf9|\x82H\xba\xc4|\xc3\xac\xf9\x91[\xa6\x0b]v\xb0\xe8\x8f)\x02\xd7\xa7F\xff\xdd\x1c\xac\xcfL\xe7\xd7V\x06\xc9\xbe4\xbe)<\xc92\xd00\x9b\xb8\tz\xb1\xd3g\x04\x03'
<class '__main__.HEADER'>


3

### MESSAGE TO SEND

In [34]:
header = HEADER(dh_pair=dh_pair,pn=0,n=0)#34B
ad = b"Associate data"
ad = hashlib.sha256(ad).digest()#32B
enc_msg = ENCRYPT(mk=mk, plaintext=b"Hola mundo!",assotiate_data=CONCAT(ad, header=header))
header, enc_msg

(<__main__.HEADER at 0x7f40dbc8a290>,
 b'\xd4h\xfc\xa4SX\xb9\xdbX0~Z\x83r\xbe\xcehQ\xceF[\xa5k\\#\x1c~\x95\xfb/!\xb5\xfb\xb8\x1c.K\xdc\xb4T\xb8p\xb4C\x95\x1c\x16l\xf8\x87\xf2\x10u\x13\xf9\xc1\x9d\x8a\xaf\rTe1\xe5\x8f\xe2rK\xdc\xe4\x01\xec\xc1\xc0\x7f|y\xf6F .\xb1D\x00#\xf1\xd6\xebTj\x87\x0bD\xabv:')

### DECRYPT

In [36]:
def DECRYPT(mk, ciphertext, assotiate_data):
    tag = ciphertext[:16]
    ad_original = ciphertext[16:16+64]
    ciphertext = ciphertext[16+64:]

    # Key Derivation Function
    output_length = 80
    prk = hkdf_extract(salt=bytes(output_length), input_key_material=mk, hash=hashlib.sha512)
    exp = hkdf_expand(pseudo_random_key=prk, info=b"Info ENCRYPT/DECRYPT", length=output_length, hash=hashlib.sha512)

    sk = exp[:32] # Encryption Key
    ak = exp[32:64] # Authentication Key
    iv = exp[64:] # Initialization vector

    decryptor = Cipher(
        algorithms.AES(key=sk),
        modes.GCM(initialization_vector=iv, tag=tag),
        backend=default_backend()
    ).decryptor()

    # Decryption and unpadding.
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(plaintext) + unpadder.finalize()

    # HMAC for authentication of the associate it data.
    ad = hmac.new(key=ak, msg=assotiate_data, digestmod=hashlib.sha512).digest() # 512b = 64B

    if ad != ad_original:
        raise Exception("Error authenticating data.")
    
    return plaintext

DECRYPT(mk=mk, ciphertext=enc_msg, assotiate_data=CONCAT(ad=ad, header=header))

b'Hola mundo!'